<center><img src='https://drive.google.com/uc?export=view&id=1qJ8NqAZolTBQY7lN-deZ8xEsU3dlUiLz' width=200></center>


<h6><center></center></h6>

<h1>
<hr style=" border:none; height:3px;">
<center>Cours XAI</center>
    <center> LAB1 : Modèles transparents</center>
    <center> Corrections</center>
<hr style=" border:none; height:3px;">
</h1>

# Prérequis
En utilisant pip, conda ou autre, il faudra installer les modules python suivants:
* numpy
* pandas
* scipy
* scikit-learn
* matplotlib
* pygam

# Prise en main du ficher de données
Les données que nous allons utiliser sont des données qui indiquent le coût de l'assurance maladie pour plus de 1300 personnes.
Une personne est décrite par:
- son âge
- son sexe
- sa masse corporelle
- son nombre d'enfants
- si elle fume ou pas
- la région dans laquelle elle vit
- le coût de son assurance.

Nous allons dans un premier temps prendre en main le fichier de données.

In [ ]:
from pandas import *
data = read_csv(r'insurance.csv')
data.info()

In [ ]:
data.head()

On remarque:
- qu'il y a 7 colonnes, 1338 lignes
- qu'il n'y a pas de valeurs manquantes
- qu'il y a 3 colonnes catégorielles (object).

Q1.Affichez les modalités (les valeurs possibles pour chacune de ces colonnes) ainsi que l'effectif correspondant.

In [ ]:
data.groupby(["sex"]).size()

In [ ]:
data.groupby(["children"]).size()

In [ ]:
data.groupby(["region"]).size()

In [ ]:
data.groupby(["smoker"]).size()

In [ ]:
# histogramme pour features numériques
data.hist(bins=10, figsize=(10,12))

Que pouvez-vous dire sur l'équilibre du jeu de données ? Apprendre sur ce dernier va-t-il poser un problème de représentativité ?

In [ ]:
#entrez ici votre réponse entre """ """

# Régression linéaire
Nous allons appliquer dans un premier temps la régression linéaire. Cependant, elle ne prend pas correctement en compte les données catégorielles.
Nous allons devoir les transformer.

## Adaptation du jeu de données
La régression linéaire ne gère pas très bien les données catégorielles, en particulier quand elles ne sont pas ordonnées.
Classiquement, on transforme:
    - les données binaires ou à 2 modalités en 0 et 1
    - les données à au moins 3 modalités en un codage binaire de ces modalités: on se retrouve avec autant de colonnes binaires qu'il y a de modalités.
Pour les colonnes a deux modalités, on peut directement les transformer avec une méthode de conversion.

In [ ]:
def YesNoConverter(val):
    if val=="yes": return 1
    else: return 0

data["smoker"] = data["smoker"].apply(YesNoConverter)

In [ ]:
def MaleFemaleConverter(val):
    #faire la même chose ici
    if val=="male": return 0
    else: return 1

data["sex"] = data["sex"].apply(MaleFemaleConverter)

Pour la dernière colonne, nous allons utiliser la fonction  [get_dummies()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) qui fait tout le travail à notre place. Il faudra ensuite insérer les colonnes créées avec [concat](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html) et supprimer la colonne originale avec [drop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html).

In [ ]:
newcols = get_dummies(data["region"])
data = concat([newcols,data], axis=1)
data = data.drop(["region"], axis=1)
data.info()
data.head()

In [ ]:
#Ou directement.
data=get_dummies(data, prefix=['region'])
print(data.columns)
data.head()

In [ ]:
data.describe()

## Régression linéaire
Maintenant que le jeu de données est prêt, utilisez [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) de scikit learn pour obtenir un modèle linéaire.

In [ ]:
#Installing 

!pip install -U scikit-learn

In [ ]:
import sklearn
from sklearn.linear_model import LinearRegression

model1 = LinearRegression()
x,y = data.iloc[:,:-1], data.iloc[:,-1]
model1 = model1.fit(x,y)

## Interprétation
Maintenant que vous avez un modèle entraînez, mettez en oeuvre ce que nous avons vu en cours (et plus!) pour l'interpréter. Utilisez au moins les poids, la SE et le t pour chaque coefficient.

In [ ]:
import numpy as np

sse = np.sum((model1.predict(x) - y) ** 2, axis=0)

degree =  float(x.shape[0] - x.shape[1])  #degree of freedom
smpvar = sse / degree #sample variance

se = np.array(np.sqrt(np.diagonal(smpvar * np.linalg.inv(np.dot(x.T, x)))))

t = model1.coef_/se

L = []

L.append(["intercept", model1.intercept_, None, None])

labels = list(x.columns) 

for i in range(len(labels)):
    L.append([labels[i], model1.coef_[i], se[i], t[i]])

res = DataFrame.from_records(L, columns=["Column", "Weight", "SE", "t"])
print(res)
print(sse)

In [ ]:
#Tapez ici votre interpretation (que révèle le modèle?)
#Concluez sur l'utilisation de ce modèle pour ce dataset

# Arbre de régression
Nous allons à présent tester un deuxième modèle: les arbres de régression. Normalement, la plupart des algorithmes acceptent tous les types de données. Cependant, l'implémentation de scikit learn ne le permet pas. Nous allons donc conserver notre jeu de données tel que pré-traité auparavant.

J'attire sur votre attention que le problème ne vient pas des colonnes binaires (à deux modalités). En revanche, pour les colonnes catégorielles à plus de 2 modalités, l'implémentation de scikit learn traiterait plutôt des valeurs ordinales, et ce n'est pas le cas des régions chez nous. Vous trouverez sur le net des tas d'astuces qui vous diront de remplacer les modalités par des nombres, etc. Elles ne me paraissent pas judicieuses dans notre cas. Aussi, lors de l'interprétation du modèle, je vous demande de prendre avec précautions les 4 colonnes "région".

## Entraînement
Entraînez un arbre de régression avec [scikit learn](https://scikit-learn.org/stable/modules/tree.html). Ne laissez pas les valeurs par défaut dans le constructeur, car sinon votre arbre sera très profond (et donc pas interprétable). Ajustez les paramètres tout en maintenant un taux de classification élevé. 
Remarque: pour cet exercice, il n'est pas nécessaire de mettre en place une cross-validation (on fait de la régression). On peut utilisé le taux de classification sur l'ensemble d'apprentissage ou un sous-ensemble, cela suffit pour cet exercice en tout cas.

In [ ]:
%matplotlib notebook

from sklearn import tree
from matplotlib import pyplot as plt

model2 = tree.DecisionTreeRegressor(max_depth=3)
model2 = model2.fit(x,y)

_ = tree.plot_tree(model2)






In [ ]:
#ou bien
%matplotlib notebook

from sklearn import tree
from matplotlib import pyplot as plt

clf = tree.DecisionTreeRegressor(max_depth = 5, min_samples_leaf=10, max_features=3, random_state=2905)
clf = clf.fit(x, y)

In [ ]:
plt.figure(figsize=(16, 12))
tree.plot_tree(clf, fontsize=10) 
plt.savefig("tree.png")
plt.show()

Prenez la dernière ligne du jeu de données, et retrouvez le chemin de la décision dans l'arbre avec la méthode [decision_path](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor.decision_path). L'idée est de connaître la liste des tests par lesquels l'instance est passée.

In [ ]:
n_nodes = clf.tree_.node_count
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
feature = clf.tree_.feature
threshold = clf.tree_.threshold

### Chemin de décision ###

# pour le dernier exemple du jeu de données
X_test = np.array(x.loc[1337]).reshape(1, -1)
print("Example : ", X_test)
node_indicator = clf.decision_path(X_test)
leaf_id = clf.apply(X_test)

sample_id = 0
# obtain ids of the nodes `sample_id` goes through, i.e., row `sample_id`
node_index = node_indicator.indices[node_indicator.indptr[sample_id]:
                                    node_indicator.indptr[sample_id + 1]]

print('\nRules used to predict sample {id}:\n'.format(id=sample_id))
for node_id in node_index:
    # continue to the next node if it is a leaf node
    if leaf_id[sample_id] == node_id:
        continue

    # check if value of the split feature for sample 0 is below threshold
    if (X_test[sample_id, feature[node_id]] <= threshold[node_id]):
        threshold_sign = "<="
    else:
        threshold_sign = ">"

    print("decision node {node} : (X_test[{sample}, {feature}] = {value}) "
          "{inequality} {threshold})".format(
              node=node_id,
              sample=sample_id,
              feature=feature[node_id],
              value=X_test[sample_id, feature[node_id]],
              inequality=threshold_sign,
              threshold=threshold[node_id]))

## Interprétation
A présent, interprétez le modèle (intéressez-vous à la feature importance). Que révèle l'arbre?

In [ ]:
# feature importances

# get importance
importance = clf.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %s, Score: %.5f' % (x.columns[i],v))

In [ ]:
%matplotlib inline
# plot feature importance
plt.figure(figsize=(16, 6))
plt.bar(x.columns, importance, tick_label=x.columns)
plt.title("Feature importance - Decision Tree Regressor")
plt.show()

# GAM
Nous allons passer au troisième type de modèle: les GAM.

## Entraînement
En utilisant le module pyGam, entraînez un GAM pour notre tâche de régression. Ne laissez pas les paramètres par défaut.

In [ ]:
from pygam import LinearGAM, s
model3 = LinearGAM(s(4) + s(6) + s(8)).fit(x, y)
model3.summary()

Pourquoi avez-vous choisi cette architecture de GAM?

## Interprétation
Visualisez les termes de votre GAM.

In [ ]:
for i, term in enumerate(model3.terms):
    if term.isintercept:
        continue

    XX = model3.generate_X_grid(term=i)
    pdep, confi = model3.partial_dependence(term=i, X=XX, width=0.95)

    plt.figure()
    plt.plot(XX[:, term.feature], pdep)
    plt.plot(XX[:, term.feature], confi, c='r', ls='--')
    plt.title(repr(term))
    plt.show()

Quelles informations tirez-vous du GAM ?

In [ ]:
#Votre réponse ici entre """ """

# Conclusion
Indiquez ce que les différents modèles vous ont apporté comme connaissances sur les données et le fonctionnement de ces modèles.

In [ ]:
#Votre réponse ici entre """ """